In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# Base URL
base_url = "https://www.lowes.com/pl/fall-decorations/fall-wreaths-garland/1614047588"

# Create an array of URLs with offsets of 24 up to 144
offsets = range(0, 145, 24)  # Generates 0, 24, 48, ..., 144
urls = [f"{base_url}?offset={offset}" for offset in offsets]

# Initialize page count
page_count = 0

for url in urls:
    # Setting up Chrome options for Selenium within the loop
    options = Options()
    options.headless = True  # Run headless for automation environments
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36')

    # Set up the Chrome driver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Navigate to the page
    driver.get(url)

    # Wait for the page to load completely
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
    time.sleep(10)  # Additional wait time to ensure all scripts and content have loaded

    # Extract the entire HTML of the page
    html_content = driver.page_source

    # Save the HTML content to a file
    with open(f'page_{page_count}.html', 'w', encoding='utf-8') as f:
        f.write(html_content)

    print(f"Saved page {page_count} from URL: {url}")  # Log the saved page info

    # Prepare for the next page
    page_count += 1

    # Clean up: close the browser window for each URL
    driver.quit()



Saved page 0 from URL: https://www.lowes.com/pl/fall-decorations/fall-wreaths-garland/1614047588?offset=0
Saved page 1 from URL: https://www.lowes.com/pl/fall-decorations/fall-wreaths-garland/1614047588?offset=24
Saved page 2 from URL: https://www.lowes.com/pl/fall-decorations/fall-wreaths-garland/1614047588?offset=48
Saved page 3 from URL: https://www.lowes.com/pl/fall-decorations/fall-wreaths-garland/1614047588?offset=72
Saved page 4 from URL: https://www.lowes.com/pl/fall-decorations/fall-wreaths-garland/1614047588?offset=96
Saved page 5 from URL: https://www.lowes.com/pl/fall-decorations/fall-wreaths-garland/1614047588?offset=120
Saved page 6 from URL: https://www.lowes.com/pl/fall-decorations/fall-wreaths-garland/1614047588?offset=144


In [8]:
"delete file 'extracted_links.txt' is exists"

!python3 scrapy_get_product_links.py

2024-11-16 15:56:21 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-11-16 15:56:21 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.10.0 (v3.10.0:b494f5935c, Oct  4 2021, 14:59:19) [Clang 12.0.5 (clang-1205.0.22.11)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform macOS-13.0.1-arm64-arm-64bit
2024-11-16 15:56:21 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-16 15:56:21 [py.warnings] WARNING: /Users/akshaysharma/Desktop/Daash/myenv/lib/python3.10/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Sc

In [5]:
!python3 scrapy_get_product_details.py

2024-11-16 20:05:33 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-11-16 20:05:33 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.12.9, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.10.0, Python 3.10.0 (v3.10.0:b494f5935c, Oct  4 2021, 14:59:19) [Clang 12.0.5 (clang-1205.0.22.11)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.3, Platform macOS-13.0.1-arm64-arm-64bit
2024-11-16 20:05:33 [scrapy.addons] INFO: Enabled addons:
[]
2024-11-16 20:05:33 [py.warnings] WARNING: /Users/akshaysharma/Desktop/Daash/myenv/lib/python3.10/site-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Sc

In [23]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV data into a DataFrame
df = pd.read_csv('product_details.csv')

# Drop rows where the price is 0
df = df[df['Product Price'] != 0]
df

,Product Model Number,Product Name,Product Price
0,HC7-104-24W,National Tree Company,49.98
1,RAHV-DA102W-1,National Tree Company,44.88
2,2005500067,Glitzhome,40.46
3,35762377,Northlight,62.98
4,7370,Worth Imports,29.98
...,...,...,...
84,W1248,Nearly Natural,55.65
85,W1153,Nearly Natural,56.48
86,W1226,Nearly Natural,70.02
88,W1239,Nearly Natural,129.00


In [16]:
# Count wreaths by brand
wreaths_by_brand = df['Product Name'].value_counts()
wreaths_by_brand

Product Name
Nearly Natural                                                                                                                                                      24
Worth Imports                                                                                                                                                       15
Northlight                                                                                                                                                          12
National Tree Company                                                                                                                                                9
Sullivans                                                                                                                                                            8
Glitzhome                                                                                                                                               

In [18]:
# Find the price range for wreaths
print(f"Price range for wreaths: ${df['Product Price'].min()} - ${df['Product Price'].max()})")


Price range for wreaths: $19.27 - $174.7)


In [19]:
# Plotting the histogram of product prices
plt.figure(figsize=(10, 6))
plt.hist(df['Product Price'], bins=15, color='blue', edgecolor='black')
plt.title('Histogram of Product Prices')
plt.xlabel('Price ($)')
plt.ylabel('Number of Products')
plt.grid(True)
plt.show()


NameError: name 'plt' is not defined